# V4: True two layers(two pins on the different layer)

In [1]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
filepath_out='../output/bench5.router'
netlist_file_path = '../benchmark/bench5.nl'
gridfile_path  = '../benchmark/bench5.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_grid_original,layer2_grid_original = parse_gridfile(gridfile_path)

bend_penalty: 10 
 via_penalty: 20
size of layer1: (127, 317)
size of layer2: (127, 317)


In [24]:
# plot problem 根据grid文件可视化障碍物和每个格子的cost，同时标出source和target，可以显示两层
plot_problem("../output/bench5_problem.jpg",columns,rows,layer1_grid_original,layer2_grid_original,nets)

In [25]:
def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid, path):
    #print(path)
    if path:               # 判断是否为空 如果path为None 后面不可迭代会报错 5.22
        for cell in path:
            x, y, _ = cell
            layer1_grid[x][y] = -1

In [26]:
def get_cell_cost(layer_grid, cell,path_tmp,bend_penalty,via_penalty):  
    x, y, layer = cell
    cell_cost = 1  # 默认的单元代价
    if layer_grid[x][y] == -1:
        cell_cost = float('inf')  # -1表示无法通过的细胞
    elif layer_grid[x][y] != 1:
        cell_cost = layer_grid[x][y]  # 非单元代价
    if len(path_tmp) >= 2:
        prev_cell = path_tmp[-2]
        prev_x, prev_y, prev_layer = prev_cell
        if layer != prev_layer:
            cell_cost += via_penalty
        elif (prev_x != x and prev_y != y): 
            cell_cost += bend_penalty
    return cell_cost

# modified by junjun
def expand_source_to_target(rows, columns, layer1_grid,layer2_grid, source, target,bend_penalty,via_penalty):#5.22 把layer2——grid输入，计算cost加入条件判断(woods5/22)
    wavefront = {}
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    wavefront[source_tuple] = 0
    costs[source_tuple] = 0  # Initial cost for the source cell is 0

    while wavefront:
        # get lowest cost cell on a wavefront structure
        current_cell = sorted(wavefront.items(),key=lambda s:int(s[1]))[0][0]

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,costs[current_cell]

        neighbors = get_neighbors(rows, columns, current_cell,target_tuple)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])

            if neighbor_tuple not in visited:
                path_tmp= reconstruct_path(source_tuple, current_cell, parents)
                # Calculate the cost to reach the neighbor cell
                if(neighbor_tuple[2]== 1):
                  cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                if(neighbor_tuple[2]== 2):
                  cost = costs[current_cell] + get_cell_cost(layer2_grid, neighbor_tuple,path_tmp,bend_penalty,via_penalty)
                # ignore blocks
                if cost!= np.inf:
                    if neighbor_tuple not in wavefront.keys() or costs[neighbor_tuple] > cost:
                        costs[neighbor_tuple] = cost
                        parents[neighbor_tuple] = current_cell

                    if neighbor_tuple not in wavefront.keys():
                        # add cell N to waveform, indexed by pathcost
                        wavefront[neighbor_tuple]=cost         
                    else:
                        wavefront[neighbor_tuple] = min(wavefront[neighbor_tuple], cost)

        visited.add(current_cell)    
        del wavefront[current_cell]                  
    return None,None

def get_neighbors(rows, columns, cell,target_tuple):  
    x, y, layer = cell
    _,_,target_layer = target_tuple
    neighbors = []
    if (layer == 1) and (layer!=target_layer):
        neighbors.append({'x': x, 'y': y, 'layer': 3-layer})  
        if y < rows - 1:
            neighbors.append({'x': x, 'y': y + 1, 'layer': layer})
        if y > 0:
            neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
        if x < columns - 1:
            neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
        if x > 0:
            neighbors.append({'x': x - 1, 'y': y, 'layer': layer})     
    elif (layer == 2) and (layer!=target_layer):
        neighbors.append({'x': x, 'y': y, 'layer': 3-layer})  
        if x < columns - 1:
            neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
        if x > 0:
            neighbors.append({'x': x - 1, 'y': y, 'layer': layer})  
        if y < rows - 1:
            neighbors.append({'x': x, 'y': y + 1, 'layer': layer})
        if y > 0:
            neighbors.append({'x': x, 'y': y - 1, 'layer': layer})    
    elif layer == 1:
        if y < rows - 1:
            neighbors.append({'x': x, 'y': y + 1, 'layer': layer})
        if y > 0:
            neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
        if x < columns - 1:
            neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
        if x > 0:
            neighbors.append({'x': x - 1, 'y': y, 'layer': layer}) 
        neighbors.append({'x': x, 'y': y, 'layer': 3-layer})   
    else:
        if x < columns - 1:
            neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
        if x > 0:
            neighbors.append({'x': x - 1, 'y': y, 'layer': layer}) 
        if y < rows - 1:
            neighbors.append({'x': x, 'y': y + 1, 'layer': layer})
        if y > 0:
            neighbors.append({'x': x, 'y': y - 1, 'layer': layer})    
        neighbors.append({'x': x, 'y': y, 'layer': 3-layer})                                    
    return neighbors

In [27]:
def true_two_layer_router(rows, columns, layer1_grid,layer2_grid, nets,bend_penalty,via_penalty):#修改了函数名，把layer2——grid输入(woods5/22)
    routing_table = {}
    costs_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = -1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = -1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = -1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = -1
            #这里区分不同层的pin，if语句有点多(woods5/22)
        ## 防止布线在后续的pin上，先将所有的pin标记为-1；
    
    for net in nets:#在V4版本中不能分开处理
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        if pin1['layer'] == 1:
            layer1_grid[pin1['x']][pin1['y']] = 1
        if pin1['layer'] == 2:
            layer2_grid[pin1['x']][pin1['y']] = 1
        if pin2['layer'] == 1:
            layer1_grid[pin2['x']][pin2['y']] = 1
        if pin2['layer'] == 2:
            layer2_grid[pin2['x']][pin2['y']] = 1
        print('Routing net:',net_id)
        path,costs = expand_source_to_target(rows, columns, layer1_grid,layer2_grid, pin1, pin2,bend_penalty,via_penalty) 
        
        if path is not None:
            mark_path_on_grid(layer1_grid,path)
            routing_table[net_id] = path
            costs_table[net_id] = costs
    return routing_table,costs_table

In [28]:
layer1_grid = layer1_grid_original.copy()
layer2_grid = layer2_grid_original.copy()

# 距离近的优先
nets.sort(key=lambda s: ((s["pin1"]['x']-s["pin2"]['x'])**2+(s["pin1"]['y']-s["pin2"]['y'])**2))
routing_table,costs_table=true_two_layer_router(rows, columns, layer1_grid.T,layer2_grid.T, nets,bend_penalty,via_penalty)
plot_path('../output/bench5_v4.jpg',columns=columns,rows=rows,grid1=layer1_grid_original,grid2=layer2_grid_original,path_dict=routing_table) 
generate_output_file(filepath_out,net_num,routing_table)

Routing net: 121
Routing net: 123
Routing net: 68
Routing net: 72
Routing net: 75
Routing net: 92
Routing net: 13
Routing net: 62
Routing net: 35
Routing net: 87
Routing net: 76
Routing net: 11
Routing net: 40
Routing net: 82
Routing net: 22
Routing net: 37
Routing net: 5
Routing net: 48
Routing net: 81
Routing net: 114
Routing net: 1
Routing net: 19
Routing net: 23
Routing net: 49
Routing net: 125
Routing net: 122
Routing net: 10
Routing net: 100
Routing net: 69
Routing net: 51
Routing net: 106
Routing net: 89
Routing net: 8
Routing net: 63
Routing net: 47
Routing net: 70
Routing net: 107
Routing net: 124
Routing net: 31
Routing net: 111
Routing net: 12
Routing net: 32
Routing net: 42
Routing net: 66
Routing net: 2
Routing net: 44
Routing net: 99
Routing net: 64
Routing net: 96
Routing net: 104
Routing net: 109
Routing net: 24
Routing net: 43
Routing net: 74
Routing net: 54
Routing net: 84
Routing net: 61
Routing net: 120
Routing net: 73
Routing net: 98
Routing net: 115
Routing net: 1

In [29]:
costs_table

{121: 3,
 123: 3,
 68: 27,
 72: 27,
 75: 27,
 92: 37,
 13: 29,
 62: 8,
 35: 31,
 87: 31,
 76: 44,
 11: 35,
 40: 36,
 82: 48,
 22: 39,
 37: 39,
 5: 18,
 48: 29,
 81: 29,
 114: 29,
 1: 40,
 19: 40,
 23: 40,
 49: 45,
 125: 33,
 122: 34,
 10: 43,
 100: 35,
 69: 59,
 51: 53,
 106: 49,
 89: 50,
 8: 51,
 63: 51,
 47: 54,
 70: 85,
 107: 53,
 124: 46,
 31: 50,
 111: 60,
 12: 81,
 32: 81,
 42: 91,
 66: 51,
 2: 73,
 44: 63,
 99: 61,
 64: 52,
 96: 62,
 104: 52,
 109: 62,
 24: 93,
 43: 101,
 74: 56,
 54: 56,
 84: 119,
 61: 77,
 120: 58,
 73: 111,
 98: 109,
 115: 79,
 118: 102,
 56: 99,
 78: 126,
 59: 83,
 28: 127,
 113: 128,
 94: 100,
 38: 107,
 71: 67,
 77: 87,
 88: 107,
 33: 70,
 9: 113,
 58: 89,
 79: 108,
 112: 115,
 52: 111,
 3: 136,
 21: 116,
 30: 72,
 29: 95,
 39: 93,
 83: 138,
 97: 86,
 16: 94,
 20: 119,
 102: 139,
 53: 95,
 4: 120,
 45: 120,
 85: 140,
 7: 143,
 119: 123,
 90: 98,
 93: 98,
 86: 124,
 41: 99,
 110: 120,
 27: 150,
 60: 130,
 18: 105,
 65: 159,
 46: 109,
 57: 109,
 80: 111,
 26

In [ ]:
costs_table.sum

In [30]:
cnt = 0
for item in routing_table.keys():
    if routing_table[item]:
        cnt += 1
cnt

128